In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16,4)

In [2]:
path = 'D:\\'
os.chdir(path)
file = 'Extract_Scoring_Export_DeFacto.csv'

df_facto = pd.read_csv(file,
                 sep= ',',
                 engine='c',
                 encoding='utf-16'
                )

In [3]:
print('Lost '+str(df_facto.shape[0] - df_facto[df_facto['Country__c'].notnull()].shape[0]) + ' due to missing country')
df_facto = df_facto[df_facto['Country__c'].notnull()]

Lost 6607 due to missing country


In [4]:
path = 'D:\\'
os.chdir(path)
file = 'Extract_Scoring_Export_Orsay.csv'

df_orsay = pd.read_csv(file,
                 sep= ',',
                 engine='python',
                 encoding='utf-16',
                 error_bad_lines=False
                )

In [5]:
print('Lost '+str(df_orsay.shape[0] - df_orsay[df_orsay['Country__c'].notnull()].shape[0]) + ' due to missing country')
df_orsay = df_orsay[df_orsay['Country__c'].notnull()]

Lost 21132 due to missing country


In [6]:
# Extract unprofitables
unprofitable = pd.read_excel('unprofitable_customer_list_SFID.xlsx')
# Fill NAs
unprofitable['SubscriberKey (MKT Cloud)'].loc[(unprofitable['SubscriberKey (MKT Cloud)'].str.find('-')+1) == 1] = np.NaN
# Replace NAs with MCloud ID
unprofitable['SubscriberKey (MKT Cloud)'] = unprofitable['SubscriberKey (MKT Cloud)'].fillna(unprofitable['PersonContactId (commerce Cloud)'])
# Join unprofitables to initial dataframe
df_facto = df_facto.assign(unprofitable = pd.merge(df_facto['Id'], 
                                          unprofitable[['Returnrate (items)', 'SubscriberKey (MKT Cloud)']], 
                                          how='left',
                                          left_on = 'Id',
                                          right_on = 'SubscriberKey (MKT Cloud)'
                                         )['Returnrate (items)']
                                               )

df_orsay = df_orsay.assign(unprofitable = pd.merge(df_orsay['Id'], 
                                          unprofitable[['Returnrate (items)', 'SubscriberKey (MKT Cloud)']], 
                                          how='left',
                                          left_on = 'Id',
                                          right_on = 'SubscriberKey (MKT Cloud)'
                                         )['Returnrate (items)']
                                               )

C:\Users\jhernandez-lallement\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
class style:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

### Munging

In [8]:
# Munging Pipeline
def start_pipeline(dataf):
    dataf = dataf.copy()
    return dataf

def lower_cols(dataf):
    dataf.columns = [c.lower() for c in dataf.columns]
    dataf.columns = [c.replace(' ', '_') for c in dataf.columns]
    return dataf

def def_types(dataf):
    dataf['id'] = dataf['id'].astype(float)
    return dataf

def rename_cols(dataf):
    dataf = dataf.rename(columns={
        'id': 'sales_force_id',
        'email_opt_in_status__c': 'email_opt_in_status',
        'whitemail_opt_in_status__c': 'whitemail_opt_in_status',
        'push_opt_in_status__c': 'push_opt_in_status',
        'country__c': 'country',
        'employee_number__c': 'employee_number',
        'permanent_control_group__c': 'permanent_ctrl',
        'scoring__c': 'score',
    })
    return dataf
    
def campaign_ID(dataf):
    # Number commented are for the DeFacto campaign for these countries
    campaign_dict = {'AT': '7011i000000LmEmAAK',
                     'CH': '7011i000000LmErAAK',
                     'CZ': '7011i000000Ln8RAAS', #Orsay
                     #'CZ':'7011i000000LmF1AAK', #DeFacto
                     'DE': '7011i000000Ln8WAAS', #Orsay
                     #'DE':'7011i000000LmF6AAK', #DeFacto
                     'HR': '7011i000000LmFBAA0',
                     'HU': '7011i000000Ln8HAAS', #Orsay
                     #'HU':'7011i000000LmFGAA0', #DeFacto
                     'PL': '7011i000000Ln8MAAS', #Orsay
                     #'PL':'7011i000000LmFLAA0', #DeFacto
                     'RO': '7011i000000LmFMAA0',
                     'RS': '7011i000000LmFQAA0',
                     'SI': '7011i000000LmFVAA0',
                     'SK': '7011i000000LmFaAAK'
                    }

    dataf = dataf.assign(campaign_ID = dataf['country'].map(campaign_dict))
    return dataf

def exclude_employees(dataf):
    # Exclude only for White Mail
    print('Lost employees: ' +str(dataf.loc[~(dataf['employee_number'].isna())].shape[0]))
    dataf = dataf.loc[(dataf['employee_number'].isna())]
    return dataf.drop(columns={'employee_number', 'email'})

def exclude_unprofitables(dataf):
    print('Lost unprofitables: ' +str(dataf.loc[~(dataf.unprofitable.isna())].shape[0]))
    dataf = dataf.loc[dataf.unprofitable.isna()]
    return dataf.drop(columns={'unprofitable'})

def boolearize_opt_ins(dataf):
    # IMPROVE!!
    dataf[['email_opt_in_status', 'whitemail_opt_in_status', 'push_opt_in_status']].fillna('x', inplace=True)
    dataf['email_opt_in_status'] = (np.where(dataf['email_opt_in_status']
                                        .isin(['a']), dataf['email_opt_in_status'], '0'))
    dataf['whitemail_opt_in_status'] = (np.where(dataf['whitemail_opt_in_status']
                                        .isin(['a']), dataf['whitemail_opt_in_status'], '0'))
    dataf['push_opt_in_status'] = (np.where(dataf['push_opt_in_status']
                                        .isin(['a']), dataf['push_opt_in_status'], '0'))
    
    # Assign communication stream code
    cols = ['email_opt_in_status', 'whitemail_opt_in_status', 'push_opt_in_status']    
    dataf = dataf.assign(comm_stream_code = dataf[cols].sum(1)).drop(cols, 1)
    
    # Assign communication stream label
    # First position = Email
    # Second position = WH
    # Third position = Push
    comm_dict = {'WH_Email_Push':   'aaa', 
                 'WH_Email':        'aa0',
                 'WH_Push':         '0aa',
                 'WH':              '0a0',
                 'Email_Push':      'a0a',
                 'Email':           'a00', 
                 'Push':            '00a',
                 'None':            '000'
                }
    inv_map = {v: k for k, v in comm_dict.items()}
        
    dataf = dataf.assign(comm_stream = dataf['comm_stream_code'].map(inv_map))
    # For some reason, there is no WH_Email_Push category
    # Thus, replace them with WH_Email
    dataf.loc[dataf['comm_stream'] == 'WH_Email_Push', 'comm_stream'] = 'WH_Email'   
    return dataf.drop(columns={'comm_stream_code'})

def boolearize_permanent_ctrl(dataf):
    permanent_ctrl = dataf['permanent_ctrl'] == 1
    dataf = dataf.assign(permanent_ctrl = permanent_ctrl)
    return dataf

def sort_per_score(dataf):
    dataf = dataf.assign(white_mail = dataf.comm_stream.str.contains('WH'))
    return (dataf
            .sort_values(by=['white_mail', 'score'], ascending=False)
            .reset_index(drop=True)
            .drop(columns={'white_mail'}))

def compute_campaign_controls(dataf, percent_ctrl, lower_threshold_for_ctrl, upper_threshold_for_ctrl):
    dataf = dataf.sort_values(by='comm_stream')
    # List possible communication streams
    dict_group = dataf.comm_stream.unique()
    # Preallocate mask that will be copied on final frame
    curr_mask = pd.Series()
    print('Report for creation of Campaign Controls')
    print('')
    print(style.BLUE + 
          'Base proportion for control group size = ' 
          + style.END
          + style.BOLD 
          + str(percent_ctrl*100) + '%' 
          + style.END)
    print(style.BLUE +
          'Minimum group size to create control = ' 
          + style.END
          + style.BOLD           
          + str(lower_threshold_for_ctrl) 
          + ' customers.'
          + style.END)
    print(style.BLUE +
          'Maximum group size for control = ' 
          + style.END
          + style.BOLD          
          + str(upper_threshold_for_ctrl) + ' customers.'
          + style.END)
    print()
    # Loop over communication stream
    # This is suboptimal. Use groupby+apply but so far I did not manage to make it work
    for i in range(0, len(dict_group)):
        curr_comm_stream = dataf.loc[dataf['comm_stream'] == dict_group[i]]
        iMask = mask(curr_comm_stream, 
                     percent_ctrl = percent_ctrl,
                     lower_threshold_for_ctrl=lower_threshold_for_ctrl, 
                     upper_threshold_for_ctrl=upper_threshold_for_ctrl)
        curr_mask = curr_mask.append(iMask)
    # Sort the index to allocate the mask to dataf
    curr_mask = curr_mask.astype(int).sort_index()
    # Assign Campaign Control column
    dataf = dataf.assign(campaign_ctrl = curr_mask == 1)
    # Replace None since SalesForce needs empty cells
    dataf.comm_stream = dataf.comm_stream.str.replace('None', '')
    return dataf   


def ceil_number_WH(dataf, country, ceil):
    
    number_of_customers_to_nullify = dataf.loc[dataf.comm_stream.str.contains('WH')].shape[0] - ceil
    selected_WH = (dataf
                    .loc[dataf.comm_stream.str.contains('WH')]
                    
                    .iloc[-number_of_customers_to_nullify:]
                    .assign(comm_stream = dataf.comm_stream.str
                            .replace('WH_', '')
                            .replace('WH', '')
                            #.replace('_Push', 'Push')
                           )
                    )
    dataf.update(selected_WH)
    print('Selected the best ' + str(ceil) + ' customers for WH for country = '+ str(country))
    return dataf
    
def reorganize_columns(dataf):    
    return dataf[['sales_force_id', 
                  'campaign_ID', 
                  'comm_stream', 
                  'permanent_ctrl',
                  'campaign_ctrl',
                  'country',
                  'score'
                 ]].reset_index(drop=True)



    

In [9]:
def mask(dataf, percent_ctrl = 0.01, lower_threshold_for_ctrl=300, upper_threshold_for_ctrl=1000):

    # Set control size to percentage of total
    ctrl_size = (np.ceil(percent_ctrl*dataf.shape[0])).astype(int)
    # Set control size to maximum of 1000 if we get more than that
    if ctrl_size >= upper_threshold_for_ctrl: 
            ctrl_size = int(upper_threshold_for_ctrl)    
            print('Too many customers in control ' + str(dataf['comm_stream'].iloc[0]) + '. '
            + 'Control size = ' + str(ctrl_size))
            print('')
    # Set control size to 0 if we have not enough customers to compute control group
    elif ctrl_size < lower_threshold_for_ctrl:
            ctrl_size = int(0)
#         mask_ctrl = np.zeros(dataf.shape[0]) # No controls if less than X individuals
            (print('Not enough customers in control ' + str(dataf['comm_stream'].iloc[0]+'. ')
            + 'Only '+ str(dataf.shape[0]) + ' customers for ' +  str(dataf['comm_stream'].iloc[0])))
            print('')
    else:        
        print('Between ' + str(lower_threshold_for_ctrl) 
          +  ' and ' + str(upper_threshold_for_ctrl) + ' customers in control '  +  str(dataf['comm_stream'].iloc[0]) + '. '
        + str(percent_ctrl*100) + '% taken as ctrl size (' + str(ctrl_size) + ' customers).')
        print('')
        
    experiment_size = (dataf.shape[0] - ctrl_size)#.astype(int)
    # Create final binary mask, and use index from dataf for later merge
    mask_ctrl =  pd.Series(np.random.permutation(np.concatenate([np.ones(ctrl_size,
                                                                   #dtype=np.int64
                                                                        ),
                                                                 np.zeros(experiment_size,
                                                                   #dtype=np.int64
                                                                   )])),
                          index=dataf.index)
    return mask_ctrl

### Run pipeline

In [10]:
# Dictionnary stating the number of white mails per country, including controls!
number_WH_country = {
    'AT': 18000,
    'CH': 0,
    'CZ':41660,
    'HU':22400,
    'DE':41660,
    'PL':41600,
    'SK':12500,
    'RO':11500,
    'HR':6000,
    'RS':6000,
    'SI':6000
                    }

# Dictionnary stating the percent of customers that should be put in control group
percent_control_country = {
    'AT': 0.13,
    'CH': 0.03,
    'CZ': 0.11,#0.07,
    'HU': 0.30,#0.07,
    'DE': 0.11,#0.07,
    'PL': 0.11,#0.07,
    'SK': 0.14,
    'RO': 0.15,
    'HR': 0.18,
    'RS': 0.18,
    'SI': 0.2    
    
}

In [11]:
np.array([v for k,v in number_WH_country.items()]).sum()

207320

In [12]:
list_country = ['DE', 'PL', 'HU', 'CZ']

In [25]:
df_final_facto = pd.DataFrame()
#df_final_orsay = pd.DataFrame()
for i in df_facto.Country__c.unique():#list_country:#
    df_curr = df_facto.loc[df_facto.Country__c == i]
    df_clean = (df_curr
    .pipe(start_pipeline)        # Make a copy
    .pipe(lower_cols)            # Lower case
    .pipe(rename_cols)           # Rename for comprehension
    .pipe(campaign_ID)           # Add campaign ID
    .pipe(exclude_employees)     # Exclude employees
    .pipe(exclude_unprofitables)
    .pipe(boolearize_opt_ins)    # Make booleans with opt-ins
    .pipe(boolearize_permanent_ctrl)
    .pipe(sort_per_score)        # Sort
    .pipe(ceil_number_WH, 
          ceil=[v for k,v in number_WH_country.items() if k == i][0], country=i)
    .pipe(compute_campaign_controls, 
          percent_ctrl=[v for k,v in percent_control_country.items() if k == i][0], 
          lower_threshold_for_ctrl=0, 
          upper_threshold_for_ctrl=2000)
    .pipe(reorganize_columns)            
    )
    df_final_facto = df_final_facto.append(df_clean)

Lost employees: 5642
Selected the best 41600 customers for WH for country = PL
Report for creation of Campaign Controls

Base proportion for control group size = 11.0%
Minimum group size to create control = 0 customers.
Maximum group size for control = 2000 customers.

Too many customers in control . Control size = 2000

Too many customers in control Email. Control size = 2000

Between 0 and 2000 customers in control Email_Push. 11.0% taken as ctrl size (327 customers).

Too many customers in control None. Control size = 2000

Too many customers in control Push. Control size = 2000

Between 0 and 2000 customers in control WH. 11.0% taken as ctrl size (1486 customers).

Too many customers in control WH_Email. Control size = 2000

Between 0 and 2000 customers in control WH_Push. 11.0% taken as ctrl size (766 customers).

Lost employees: 2116
Selected the best 41660 customers for WH for country = CZ
Report for creation of Campaign Controls

Base proportion for control group size = 11.0%
M

Too many customers in control Email. Control size = 2000

Between 0 and 2000 customers in control None. 18.0% taken as ctrl size (73 customers).

Between 0 and 2000 customers in control Push. 18.0% taken as ctrl size (136 customers).

Between 0 and 2000 customers in control WH. 18.0% taken as ctrl size (177 customers).

Between 0 and 2000 customers in control WH_Email. 18.0% taken as ctrl size (873 customers).

Between 0 and 2000 customers in control WH_Push. 18.0% taken as ctrl size (32 customers).

Lost employees: 203
Selected the best 6000 customers for WH for country = RS
Report for creation of Campaign Controls

Base proportion for control group size = 18.0%
Minimum group size to create control = 0 customers.
Maximum group size for control = 2000 customers.

Too many customers in control . Control size = 2000

Too many customers in control Email. Control size = 2000

Between 0 and 2000 customers in control None. 18.0% taken as ctrl size (134 customers).

Between 0 and 2000 custome

In [26]:
temp = df_final_facto.loc[~(df_final_facto.unprofitable.isna())]

In [27]:
df_final_facto.comm_stream.str.contains('WH').sum()

207320

In [29]:
temp.loc[temp.comm_stream.str.contains('WH')].to_csv('unprofitables_whitemail_DressSpecial.csv')

In [30]:
pwd

'D:\\'

In [36]:
df_merged = (pd.merge(df_final_facto[['sales_force_id','comm_stream', 'score']].rename(columns={'comm_stream':'comm_stream_facto', 
                                                                                               'score':'score_facto'}), 
                     df_final_orsay, 
                     how='outer', 
                     on='sales_force_id', 
                     indicator=True
                    )
             .rename(columns={'_merge': 'source'})
             .replace(['right_only','left_only', ''], ['orsay_only','defacto_only', np.NaN])
             #
            )

In [46]:
df_merged.to_csv('overall_data_for_campaign_valuation.csv')

In [38]:
df_facto_only = df_merged.loc[(df_merged['source'] == 'defacto_only')]

In [127]:
df_facto_only.comm_stream_facto.str.contains('WH')

5710

In [81]:
df_orsay_only = df_merged.loc[(df_merged['source'] == 'orsay_only')]

In [131]:
df_all_facto = (df_merged
                     .loc[(df_merged['source'] == 'facto_only') | (df_merged['source'] == 'both')]
                     .dropna(subset=['comm_stream_facto'])
                      )

In [37]:
df_orsay_with_facto = (df_merged
                     .loc[(df_merged['source'] == 'orsay_only') | (df_merged['source'] == 'both')]
                     .dropna(subset=['comm_stream'])
                      )

In [38]:
df_selected_WH = df_orsay_with_facto.loc[df_orsay_with_facto.comm_stream.str.contains('WH')]

In [39]:
df_orsay_campaign = df_selected_WH.loc[df_selected_WH.comm_stream_facto.isna()]

In [40]:
df_final_orsay = df_orsay_campaign.loc[ 
                       (df_orsay_campaign.country == 'DE')
                      |(df_orsay_campaign.country == 'PL')
                      |(df_orsay_campaign.country == 'HU')
                      |(df_orsay_campaign.country == 'CZ')].drop(columns={'comm_stream_facto','score_facto', 'source'})
#.groupby('country').count()

In [48]:
df_final_orsay.groupby(['country', 'campaign_ctrl']).count().sales_force_id

country  campaign_ctrl
CZ       False            7800
         True              941
DE       False            8276
         True              948
HU       False            2449
         True              868
PL       False            8019
         True              897
Name: sales_force_id, dtype: int64

In [42]:
df_final_orsay.comm_stream = 'WH'

In [43]:
df_final_orsay

,sales_force_id,campaign_ID,comm_stream,permanent_ctrl,campaign_ctrl,country,score
6960,0031i000004uxKDAAY,7011i000000Ln8MAAS,WH,False,False,PL,0.279113
13127,0031i00000400dcAAA,7011i000000Ln8MAAS,WH,False,False,PL,0.241904
40856,0031i000004wCr9AAE,7011i000000Ln8MAAS,WH,False,False,PL,0.418322
45868,00358000016a1NxAAI,7011i000000Ln8MAAS,WH,False,False,PL,0.390619
55901,0035800000kmk2cAAA,7011i000000Ln8MAAS,WH,False,False,PL,0.254644
...,...,...,...,...,...,...,...
8582145,0035800000yrgTTAAY,7011i000000Ln8HAAS,WH,False,False,HU,0.312655
8582146,0035800000yrJ1qAAE,7011i000000Ln8HAAS,WH,False,False,HU,0.257586
8582147,0031i00000J5gXRAAZ,7011i000000Ln8HAAS,WH,False,True,HU,0.203875
8582148,0035800000yrdC1AAI,7011i000000Ln8HAAS,WH,False,False,HU,0.315886


In [44]:
filename = 'file_campaign_dress_special_2020_Orsay'

In [45]:
# Save campaign file
df_final_orsay.to_csv(filename+'.csv', index=False)

In [46]:
# Overall Descriptive Table with all values
descriptive_table = pd.DataFrame(df_final_orsay.groupby(['country',
                                                   'comm_stream', 
                                                   #'permanent_ctrl',
                                                   'campaign_ctrl'
                                                  ])
                                 .describe()['score']['count'])    

descriptive_table.to_csv(filename+'_descriptive.csv')

In [47]:
# Descriptive Table with structured values
temp=pd.DataFrame(df_final_orsay
                   .groupby(['country',
                             'comm_stream', 
                             #'permanent_ctrl',
                             'campaign_ctrl'
                            ])
                   .describe()['score']['count']
            )
campaign_false = (temp
.reset_index(level=['campaign_ctrl', 'comm_stream'])
.replace('', np.NaN)
.dropna()
.reset_index()
.groupby(['country', 'comm_stream'])#['count']
.first()
.rename(columns={'count': 'value_target'})
.drop(columns={'campaign_ctrl'})
)
campaign_true = (temp
.reset_index(level=['campaign_ctrl', 'comm_stream'])
.replace('', np.NaN)
.dropna()
.reset_index()
.groupby(['country', 'comm_stream'])#['count']
.last()
.rename(columns={'count': 'value_ctrl'})
.drop(columns={'campaign_ctrl'})
)

pd.concat([campaign_false, campaign_true],axis=1).to_csv(filename+'_descriptive_structured.csv')

In [75]:
country='DE'
temp=descriptive_table.loc[(country)].reset_index()
print(str(country))
stream = 'Push'
idx = temp.comm_stream.str.contains(stream)
print(str(stream)+': ' +str(temp.loc[idx]['count'].sum()))
stream = 'Email'
idx = temp.comm_stream.str.contains(stream)
print(str(stream)+': ' +str(temp.loc[idx]['count'].sum()))
stream = 'WH'
idx = temp.comm_stream.str.contains(stream)
print(str(stream)+': ' +str(temp.loc[idx]['count'].sum()))

DE
Push: 0.0
Email: 0.0
WH: 9224.0
